In [17]:
import pyvisa

# ───── Setup VISA ─────
rm   = pyvisa.ResourceManager()
inst = rm.open_resource("TCPIP0::192.168.1.155::5025::SOCKET")  # ← your address

inst.timeout           = 5000       # 5 s for big transfers
inst.chunk_size        = 10**6      # 1 MB chunks
inst.write_termination = '\n'
inst.read_termination  = '\n'
# ───── Milestone 1 ─────
inst.write("CHDR OFF")           # suppress text headers
inst.write("WFSU SP,0,NP,0,FP,0") # full‐buffer, no sparsing
inst.write("C1:WF? DAT2")        # ask for raw waveform

# 1) Skip any bytes until we see the block marker '#'
while True:
    start = inst.read_bytes(1)
    if start == b'#':
        break

# 2) Next byte is ASCII digit D = number of length‐digits
digit_byte = inst.read_bytes(1)
num_digits = int(digit_byte)     # e.g. b'9' → 9

# 3) Read exactly those many ASCII digits for the payload length
length_str = inst.read_bytes(num_digits)         # e.g. b'000000070'
data_len   = int(length_str.decode('ascii'))     # → 70

# 4) Read the payload of that exact length
payload    = inst.read_bytes(data_len)

# 5) Read the trailing terminator (usually \n\n)
terminator = inst.read_bytes(2)

# 6) Stitch it all back together
raw_block = b'#' + digit_byte + length_str + payload + terminator

# 7) Show it for byte-by-byte comparison with the manual
print(repr(raw_block))

print(payload)

# Clean up
inst.close()
rm.close()


b'#9000070000\xff\xff\xff\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfe\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfe\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfd\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfc\xfb\xfb\xfb\xfb\xfb\xfb\xfb\xfc\xfb\xfb\xfb\xfb\xfa\xfb\xfb\xfb\xfb\xfb\xfb\xfa\xfa\xfa\xfa\xfa\xfa\xfa\xfb\xfb\xfa\xfa\xfa\xfa\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf8\xf7\xf7\xf7\xf7\xf8\xf8\xf8\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf6\xf6\xf6\xf6\xf6\xf7\xf7\xf7\xf7\xf7\xf7\xf7\xf6\xf6\xf5\xf5\xf6\xf6\xf7\xf7\xf6\xf6\xf6\xf6\xf6\xf5\xf5\xf5\xf5\xf6\xf6\xf6\xf6\xf5\xf5\xf5\xf5\xf5\xf5\xf5\xf5\xf5\xf5\xf5\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf4\xf3\xf3\xf2\xf2\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf3\xf2\xf2\xf2\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf1\xf

In [25]:
# `payload` is the bytes object you read from the scope
# e.g. payload = b'\xff\xff\xfe\xfd…'

# Method A: simple list comprehension
codes = [(b - 256) if b > 127 else b for b in payload]

# Method B: using the array module (auto sign-conversion)


# Inspect the first few
print(codes[69900:])

[4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
